<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkDeltapartitionID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SparkDeltaOptimize") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [3]:
df=spark.read.format('csv').options(header=True,inferSchema=True).load('/content/drive/MyDrive/Abc/SalesData/sales_data.csv')
df.show()

+--------------+----------+-----------+----------+------+--------+--------------------+
|transaction_id|product_id|customer_id| sale_date|amount|currency|              region|
+--------------+----------+-----------+----------+------+--------+--------------------+
|          1001|      P001|       C101|2024-01-15|150.75|     USD|               North|
|          1002|      P002|       C102|2024-01-15| 200.0|     EUR|               South|
|          1003|      P003|       C103|2024-01-16| 50.25|     USD|                East|
|          1004|      P001|       C101|2024-01-16| 300.5|     USD|               North|
|          1005|      P004|       C104|2024-01-17|  75.0|     GBP|                West|
|          1006|      P002|       C105|2024-01-17| 120.0|     EUR|               South|
|          1007|      P005|       C101|2024-01-18| 99.99|     USD|               North|
|          1008|      P001|       C102|2024-01-18|  25.0|     USD|               South|
|          1009|      P003|     

As the Uploaded data is small, Spark by default created single partiton and loaded the data.

In [4]:
df.rdd.getNumPartitions()

1

to get the data which is resided in different  partions in  dataframe, we can use spark_partition_id fucntion.

In [6]:
from pyspark.sql.functions import *
df1=df.withColumn('Partition_id',spark_partition_id())
df1.show()

+--------------+----------+-----------+----------+------+--------+--------------------+------------+
|transaction_id|product_id|customer_id| sale_date|amount|currency|              region|Partition_id|
+--------------+----------+-----------+----------+------+--------+--------------------+------------+
|          1001|      P001|       C101|2024-01-15|150.75|     USD|               North|           0|
|          1002|      P002|       C102|2024-01-15| 200.0|     EUR|               South|           0|
|          1003|      P003|       C103|2024-01-16| 50.25|     USD|                East|           0|
|          1004|      P001|       C101|2024-01-16| 300.5|     USD|               North|           0|
|          1005|      P004|       C104|2024-01-17|  75.0|     GBP|                West|           0|
|          1006|      P002|       C105|2024-01-17| 120.0|     EUR|               South|           0|
|          1007|      P005|       C101|2024-01-18| 99.99|     USD|               North|    

To understand properly manually repartitioning the data and applying spark_partiton_id() and doing a groupBy aggregate to understand the function.

In [17]:
df1=df1.repartition(24)
df1=df1.withColumn('Partition_id1',spark_partition_id())

In [18]:
print(df1.rdd.getNumPartitions())
df1.show()

24
+--------------+----------+-----------+----------+------+--------+--------------------+------------+-------------+
|transaction_id|product_id|customer_id| sale_date|amount|currency|              region|Partition_id|Partition_id1|
+--------------+----------+-----------+----------+------+--------+--------------------+------------+-------------+
|          1020|      P005|       C101|2024-03-01| 120.5|     USD|               North|           0|            0|
|          1018|      P003|       C103|2024-02-06| -10.0|     USD|                East|           0|            0|
|          1012|      P001|       C101|2024-01-20|  10.0|     USD|               North|           0|            1|
|          1032|      P002|       C102|2024-05-04|  30.0|     EUR|               South|           0|            1|
|          1043|      P003|       C103|2024-03-15|  60.0|     USD|                East|           0|            2|
|          1027|      P003|       C106|2024-04-03|  90.0|     EUR|           

In [22]:
df1.groupBy('Partition_id1').count().orderBy(desc("count")).show(25)

+-------------+-----+
|Partition_id1|count|
+-------------+-----+
|           22|    3|
|           21|    3|
|            0|    2|
|            1|    2|
|            3|    2|
|            5|    2|
|            2|    2|
|            9|    2|
|            4|    2|
|           13|    2|
|            6|    2|
|            7|    2|
|            8|    2|
|           23|    2|
|           10|    2|
|           11|    2|
|           12|    2|
|           14|    2|
|           15|    2|
|           16|    2|
|           17|    2|
|           18|    2|
|           19|    2|
|           20|    2|
+-------------+-----+



###### this helps us understand if the data is skewed or not, after the data is uploaded by spark application, in real world scenarios data might be huge and this helps us to avoid data skewness, there by optimizing the queries.

######**END OF CODE**